In [ ]:
import lib._util.visualplot as vp

# Feature scaling
from lib._class.DFStandardScaler import DFStandardScaler

In [ ]:
import numpy as np
import pandas as pd

# Imbalanced-Learn
from imblearn.pipeline import Pipeline

# Scikit-Learn
from sklearn.datasets import make_blobs

# Plotly
import plotly.express as px

In [ ]:
OUT_PATH_GRAPH = 'resources/output/graph/'

def custom_blobs(n_samples, n_classes=2, n_features=2, center_box=(-10.0, 10.0), weights=None, random_state=None):
    if weights is not None:
        assert np.sum(weights) == 1, 'sum of weights must be 1.'
        assert len(weights) == n_classes, 'no. of weights element must be same as n_classes.'
    
    # Generate balanced dataset
    X, y = make_blobs(n_samples=n_samples * n_classes, centers=n_classes, n_features=n_features,
                      center_box=center_box, random_state=random_state)
    
    data_df = to_frame(X, y)
    X = data_df.drop(columns=['target']).copy()
    y = data_df['target'].copy()
    
    if weights is None:
        return X, y
    
    # Generate imbalance dataset
    # Reference: # https://machinelearningmastery.com/how-to-develop-an-intuition-skewed-class-distributions/
    proportions = {}
    for x in range(n_classes):
        proportions[x] = int(np.round(n_samples * weights[x]))
    
    # Collect the examples
    new_X, new_y = [], []
    for k, v in proportions.items():
        indexes = np.where(y == k)[0]
        selected_indexes = indexes[:v]
        new_X.append(X[X.index.isin(selected_indexes)])
        new_y.append(y[y.index.isin(selected_indexes)])
        
    X = np.vstack(new_X)
    y = np.hstack(new_y)
    
    data_df = to_frame(X, y)
    X = data_df.drop(columns=['target']).copy()
    y = data_df['target'].copy()
    
    return X, y

def to_frame(X, y):
    data_df = pd.DataFrame(X, columns=[f'feature_{x+1}' for x in range(X.shape[1])])
    data_df['target'] = y
    
    return data_df

def class_ratio(y):
    return {k: round(v,2) for k,v in sorted(y.value_counts(normalize=True).to_dict().items())}

In [ ]:
data_groups    = []
subplot_titles = []

X, y = custom_blobs(n_samples=1000, n_classes=2, n_features=2,
                    center_box=(-10, 5), weights=[.9, .1], random_state=0)

X.shape, y.shape

In [ ]:
vp.value_count(y.to_frame(), 'target')

In [ ]:
fig = px.scatter(pd.concat([DFStandardScaler().fit_transform(X), y.astype(str)], axis=1).sort_values(by='target'),
                 x='feature_1', y='feature_2', color='target')

data_groups.append(fig['data'])
subplot_titles.append(f'Original Dataset {class_ratio(y)}')

# 1. Under-Sampling

### 1.1. Condensed Nearest Neighbour

In [ ]:
from imblearn.under_sampling import CondensedNearestNeighbour

In [ ]:
model = CondensedNearestNeighbour(n_jobs=-1, random_state=0)

steps = [
    ('standard_scaler', DFStandardScaler()),
    ('model', model),
]
new_X, new_y = Pipeline(steps, verbose=True).fit_resample(X, y)

# Removed samples
# X[X.index.isin(model.sample_indices_) == False]
# y[y.index.isin(model.sample_indices_) == False]

new_X.shape, new_y.shape

In [ ]:
vp.value_count(new_y.to_frame(), 'target')

In [ ]:
fig = px.scatter(pd.concat([new_X, new_y.astype(str)], axis=1).sort_values(by='target'),
                 x='feature_1', y='feature_2', color='target')

data_groups.append(fig['data'])
subplot_titles.append(f'Condensed Nearest Neighbour {class_ratio(new_y)}')

In [ ]:
subplot_titles

### 1.2. Edited Nearest Neighbour

In [ ]:
from imblearn.under_sampling import EditedNearestNeighbours

In [ ]:
model = EditedNearestNeighbours(n_jobs=-1)

steps = [
    ('standard_scaler', DFStandardScaler()),
    ('model', model),
]
new_X, new_y = Pipeline(steps, verbose=True).fit_resample(X, y)

# Removed samples
# X[X.index.isin(model.sample_indices_) == False]
# y[y.index.isin(model.sample_indices_) == False]

new_X.shape, new_y.shape

In [ ]:
vp.value_count(new_y.to_frame(), 'target')

In [ ]:
fig = px.scatter(pd.concat([new_X, new_y.astype(str)], axis=1).sort_values(by='target'),
                 x='feature_1', y='feature_2', color='target')

data_groups.append(fig['data'])
subplot_titles.append(f'Edited Nearest Neighbours {class_ratio(new_y)}')

### 1.3. Repeated Edited Nearest Neighbour

In [ ]:
from imblearn.under_sampling import RepeatedEditedNearestNeighbours

In [ ]:
model = RepeatedEditedNearestNeighbours(n_jobs=-1)

steps = [
    ('standard_scaler', DFStandardScaler()),
    ('model', model),
]
new_X, new_y = Pipeline(steps, verbose=True).fit_resample(X, y)

# Removed samples
# X[X.index.isin(model.sample_indices_) == False]
# y[y.index.isin(model.sample_indices_) == False]

new_X.shape, new_y.shape

In [ ]:
vp.value_count(new_y.to_frame(), 'target')

In [ ]:
fig = px.scatter(pd.concat([new_X, new_y.astype(str)], axis=1).sort_values(by='target'),
                 x='feature_1', y='feature_2', color='target')

data_groups.append(fig['data'])
subplot_titles.append(f'Repeated Edited Nearest Neighbours {class_ratio(new_y)}')

### 1.4. All KNN

In [ ]:
from imblearn.under_sampling import AllKNN

In [ ]:
model = AllKNN(n_jobs=-1)

steps = [
    ('standard_scaler', DFStandardScaler()),
    ('model', model),
]
new_X, new_y = Pipeline(steps, verbose=True).fit_resample(X, y)

# Removed samples
# X[X.index.isin(model.sample_indices_) == False]
# y[y.index.isin(model.sample_indices_) == False]

new_X.shape, new_y.shape

In [ ]:
vp.value_count(new_y.to_frame(), 'target')

In [ ]:
fig = px.scatter(pd.concat([new_X, new_y.astype(str)], axis=1).sort_values(by='target'),
                 x='feature_1', y='feature_2', color='target')

data_groups.append(fig['data'])
subplot_titles.append(f'All KNN {class_ratio(new_y)}')

### 1.5. Instance Hardness Threshold

In [ ]:
from imblearn.under_sampling import InstanceHardnessThreshold
from sklearn.ensemble import RandomForestClassifier

In [ ]:
model = InstanceHardnessThreshold(estimator=RandomForestClassifier(), n_jobs=-1, random_state=0)

steps = [
    ('standard_scaler', DFStandardScaler()),
    ('model', model),
]
new_X, new_y = Pipeline(steps, verbose=True).fit_resample(X, y)

# Removed samples
# X[X.index.isin(model.sample_indices_) == False]
# y[y.index.isin(model.sample_indices_) == False]

new_X.shape, new_y.shape

In [ ]:
vp.value_count(new_y.to_frame(), 'target')

In [ ]:
fig = px.scatter(pd.concat([new_X, new_y.astype(str)], axis=1).sort_values(by='target'),
                 x='feature_1', y='feature_2', color='target')

data_groups.append(fig['data'])
subplot_titles.append(f'Instance Hardness Threshold {class_ratio(new_y)}')

### 1.6. Near Miss - V1

In [ ]:
from imblearn.under_sampling import NearMiss

In [ ]:
model = NearMiss(version=1, n_jobs=-1)

steps = [
    ('standard_scaler', DFStandardScaler()),
    ('model', model),
]
new_X, new_y = Pipeline(steps, verbose=True).fit_resample(X, y)

# Removed samples
# X[X.index.isin(model.sample_indices_) == False]
# y[y.index.isin(model.sample_indices_) == False]

new_X.shape, new_y.shape

In [ ]:
vp.value_count(new_y.to_frame(), 'target')

In [ ]:
fig = px.scatter(pd.concat([new_X, new_y.astype(str)], axis=1).sort_values(by='target'),
                 x='feature_1', y='feature_2', color='target')

data_groups.append(fig['data'])
subplot_titles.append(f'Near Miss - V1 {class_ratio(new_y)}')

### 1.6. Near Miss - V2

In [ ]:
model = NearMiss(version=2, n_jobs=-1)

steps = [
    ('standard_scaler', DFStandardScaler()),
    ('model', model),
]
new_X, new_y = Pipeline(steps, verbose=True).fit_resample(X, y)

# Removed samples
# X[X.index.isin(model.sample_indices_) == False]
# y[y.index.isin(model.sample_indices_) == False]

new_X.shape, new_y.shape

In [ ]:
vp.value_count(new_y.to_frame(), 'target')

In [ ]:
fig = px.scatter(pd.concat([new_X, new_y.astype(str)], axis=1).sort_values(by='target'),
                 x='feature_1', y='feature_2', color='target')

data_groups.append(fig['data'])
subplot_titles.append(f'Near Miss - V2 {class_ratio(new_y)}')

### 1.6. Near Miss - V3

In [ ]:
model = NearMiss(version=3, n_jobs=-1)

steps = [
    ('standard_scaler', DFStandardScaler()),
    ('model', model),
]
new_X, new_y = Pipeline(steps, verbose=True).fit_resample(X, y)

# Removed samples
# X[X.index.isin(model.sample_indices_) == False]
# y[y.index.isin(model.sample_indices_) == False]

new_X.shape, new_y.shape

In [ ]:
vp.value_count(new_y.to_frame(), 'target')

In [ ]:
fig = px.scatter(pd.concat([new_X, new_y.astype(str)], axis=1).sort_values(by='target'),
                 x='feature_1', y='feature_2', color='target')

data_groups.append(fig['data'])
subplot_titles.append(f'Near Miss - V3 {class_ratio(new_y)}')

### 1.7. Neighbourhood Cleaning Rule

In [ ]:
from imblearn.under_sampling import NeighbourhoodCleaningRule

In [ ]:
model = NeighbourhoodCleaningRule(n_jobs=-1)

steps = [
    ('standard_scaler', DFStandardScaler()),
    ('model', model),
]
new_X, new_y = Pipeline(steps, verbose=True).fit_resample(X, y)

# Removed samples
# X[X.index.isin(model.sample_indices_) == False]
# y[y.index.isin(model.sample_indices_) == False]

new_X.shape, new_y.shape

In [ ]:
vp.value_count(new_y.to_frame(), 'target')

In [ ]:
fig = px.scatter(pd.concat([new_X, new_y.astype(str)], axis=1).sort_values(by='target'),
                 x='feature_1', y='feature_2', color='target')

data_groups.append(fig['data'])
subplot_titles.append(f'Neighbourhood Cleaning Rule {class_ratio(new_y)}')

### 1.8. One Sided Selection

In [ ]:
from imblearn.under_sampling import OneSidedSelection

In [ ]:
model = OneSidedSelection(random_state=0, n_jobs=-1)

steps = [
    ('standard_scaler', DFStandardScaler()),
    ('model', model),
]
new_X, new_y = Pipeline(steps, verbose=True).fit_resample(X, y)

# Removed samples
# X[X.index.isin(model.sample_indices_) == False]
# y[y.index.isin(model.sample_indices_) == False]

new_X.shape, new_y.shape

In [ ]:
vp.value_count(new_y.to_frame(), 'target')

In [ ]:
fig = px.scatter(pd.concat([new_X, new_y.astype(str)], axis=1).sort_values(by='target'),
                 x='feature_1', y='feature_2', color='target')

data_groups.append(fig['data'])
subplot_titles.append(f'One Sided Selection {class_ratio(new_y)}')

### 1.9. Random Under Sampler

In [ ]:
from imblearn.under_sampling import RandomUnderSampler

In [ ]:
model = RandomUnderSampler(random_state=0)

steps = [
    ('standard_scaler', DFStandardScaler()),
    ('model', model),
]
new_X, new_y = Pipeline(steps, verbose=True).fit_resample(X, y)

# Removed samples
# X[X.index.isin(model.sample_indices_) == False]
# y[y.index.isin(model.sample_indices_) == False]

new_X.shape, new_y.shape

In [ ]:
vp.value_count(new_y.to_frame(), 'target')

In [ ]:
fig = px.scatter(pd.concat([new_X, new_y.astype(str)], axis=1).sort_values(by='target'),
                 x='feature_1', y='feature_2', color='target')

data_groups.append(fig['data'])
subplot_titles.append(f'Random Under Sampler {class_ratio(new_y)}')

### 1.10. Tomek Links

In [ ]:
from imblearn.under_sampling import TomekLinks

In [ ]:
model = TomekLinks(n_jobs=-1)

steps = [
    ('standard_scaler', DFStandardScaler()),
    ('model', model),
]
new_X, new_y = Pipeline(steps, verbose=True).fit_resample(X, y)

# Removed samples
# X[X.index.isin(model.sample_indices_) == False]
# y[y.index.isin(model.sample_indices_) == False]

new_X.shape, new_y.shape

In [ ]:
vp.value_count(new_y.to_frame(), 'target')

In [ ]:
fig = px.scatter(pd.concat([new_X, new_y.astype(str)], axis=1).sort_values(by='target'),
                 x='feature_1', y='feature_2', color='target')

data_groups.append(fig['data'])
subplot_titles.append(f'Tomek Links {class_ratio(new_y)}')

In [ ]:
# Scatter plot
vp.datagroups_subplots(
    data_groups,
    max_col=3,
    title='Under-Sampling',
    out_path=OUT_PATH_GRAPH,
    subplot_kwargs={
        'subplot_titles': subplot_titles,
        'vertical_spacing': .05
    },
    layout_kwargs={'height': 1500})

# 2. Over-Sampling

In [ ]:
data_groups    = []
subplot_titles = []

fig = px.scatter(pd.concat([DFStandardScaler().fit_transform(X), y.astype(str)], axis=1).sort_values(by='target'),
                 x='feature_1', y='feature_2', color='target')

data_groups.append(fig['data'])
subplot_titles.append(f'Original Dataset {class_ratio(y)}')

### 2.1. Adaptive Synthetic (ADASYN)

In [ ]:
from imblearn.over_sampling import ADASYN

In [ ]:
model = ADASYN(random_state=0, n_jobs=-1)

steps = [
    ('standard_scaler', DFStandardScaler()),
    ('model', model),
]
new_X, new_y = Pipeline(steps, verbose=True).fit_resample(X, y)

new_X.shape, new_y.shape

In [ ]:
vp.value_count(new_y.to_frame(), 'target')

In [ ]:
fig = px.scatter(pd.concat([new_X, new_y.astype(str)], axis=1).sort_values(by='target'),
                 x='feature_1', y='feature_2', color='target')

data_groups.append(fig['data'])
subplot_titles.append(f'ADASYN {class_ratio(new_y)}')

### 2.2. Borderline SMOTE

In [ ]:
from imblearn.over_sampling import BorderlineSMOTE

In [ ]:
model = BorderlineSMOTE(random_state=0, n_jobs=-1)

steps = [
    ('standard_scaler', DFStandardScaler()),
    ('model', model),
]
new_X, new_y = Pipeline(steps, verbose=True).fit_resample(X, y)

new_X.shape, new_y.shape

In [ ]:
vp.value_count(new_y.to_frame(), 'target')

In [ ]:
fig = px.scatter(pd.concat([new_X, new_y.astype(str)], axis=1).sort_values(by='target'),
                 x='feature_1', y='feature_2', color='target')

data_groups.append(fig['data'])
subplot_titles.append(f'Borderline SMOTE {class_ratio(new_y)}')

### 2.3. K-Means SMOTE

In [ ]:
from imblearn.over_sampling import KMeansSMOTE

In [ ]:
model = KMeansSMOTE(random_state=0, n_jobs=-1)

steps = [
    ('standard_scaler', DFStandardScaler()),
    ('model', model),
]
new_X, new_y = Pipeline(steps, verbose=True).fit_resample(X, y)

new_X.shape, new_y.shape

In [ ]:
vp.value_count(new_y.to_frame(), 'target')

In [ ]:
fig = px.scatter(pd.concat([new_X, new_y.astype(str)], axis=1).sort_values(by='target'),
                 x='feature_1', y='feature_2', color='target')

data_groups.append(fig['data'])
subplot_titles.append(f'K-Means SMOTE {class_ratio(new_y)}')

### 2.4. Random Over Sampler

In [ ]:
from imblearn.over_sampling import RandomOverSampler

In [ ]:
model = RandomOverSampler(random_state=0)

steps = [
    ('standard_scaler', DFStandardScaler()),
    ('model', model),
]
new_X, new_y = Pipeline(steps, verbose=True).fit_resample(X, y)

new_X.shape, new_y.shape

In [ ]:
vp.value_count(new_y.to_frame(), 'target')

In [ ]:
fig = px.scatter(pd.concat([new_X, new_y.astype(str)], axis=1).sort_values(by='target'),
                 x='feature_1', y='feature_2', color='target')

data_groups.append(fig['data'])
subplot_titles.append(f'Random Over Sampler {class_ratio(new_y)}')

### 2.5. SMOTE

In [ ]:
from imblearn.over_sampling import SMOTE

In [ ]:
model = SMOTE(random_state=0, n_jobs=-1)

steps = [
    ('standard_scaler', DFStandardScaler()),
    ('model', model),
]
new_X, new_y = Pipeline(steps, verbose=True).fit_resample(X, y)

new_X.shape, new_y.shape

In [ ]:
vp.value_count(new_y.to_frame(), 'target')

In [ ]:
fig = px.scatter(pd.concat([new_X, new_y.astype(str)], axis=1).sort_values(by='target'),
                 x='feature_1', y='feature_2', color='target')

data_groups.append(fig['data'])
subplot_titles.append(f'SMOTE {class_ratio(new_y)}')

### 2.6. SVM SMOTE

In [ ]:
from imblearn.over_sampling import SVMSMOTE

In [ ]:
model = SVMSMOTE(random_state=0, n_jobs=-1)

steps = [
    ('standard_scaler', DFStandardScaler()),
    ('model', model),
]
new_X, new_y = Pipeline(steps, verbose=True).fit_resample(X, y)

new_X.shape, new_y.shape

In [ ]:
vp.value_count(new_y.to_frame(), 'target')

In [ ]:
fig = px.scatter(pd.concat([new_X, new_y.astype(str)], axis=1).sort_values(by='target'),
                 x='feature_1', y='feature_2', color='target')

data_groups.append(fig['data'])
subplot_titles.append(f'SVM SMOTE {class_ratio(new_y)}')

In [ ]:
# Scatter plot
vp.datagroups_subplots(
    data_groups,
    max_col=3,
    title='Over-Sampling',
    out_path=OUT_PATH_GRAPH,
    subplot_kwargs={
        'subplot_titles': subplot_titles,
        'vertical_spacing': .09
    },
    layout_kwargs={'height': 950})

# 3. Combination (Over+Under-Sampling)

In [ ]:
data_groups    = []
subplot_titles = []

fig = px.scatter(pd.concat([DFStandardScaler().fit_transform(X), y.astype(str)], axis=1).sort_values(by='target'),
                 x='feature_1', y='feature_2', color='target')

data_groups.append(fig['data'])
subplot_titles.append(f'Original Dataset {class_ratio(y)}')

### 3.1. SMOTE+ENN

In [ ]:
from imblearn.combine import SMOTEENN

In [ ]:
model = SMOTEENN(random_state=0, n_jobs=-1)

steps = [
    ('standard_scaler', DFStandardScaler()),
    ('model', model),
]
new_X, new_y = Pipeline(steps, verbose=True).fit_resample(X, y)

new_X.shape, new_y.shape

In [ ]:
vp.value_count(new_y.to_frame(), 'target')

In [ ]:
fig = px.scatter(pd.concat([new_X, new_y.astype(str)], axis=1).sort_values(by='target'),
                 x='feature_1', y='feature_2', color='target')

data_groups.append(fig['data'])
subplot_titles.append(f'SMOTE+ENN {class_ratio(new_y)}')

### 3.2. SMOTE+Tomek

In [ ]:
from imblearn.combine import SMOTETomek

In [ ]:
model = SMOTETomek(random_state=0, n_jobs=-1)

steps = [
    ('standard_scaler', DFStandardScaler()),
    ('model', model),
]
new_X, new_y = Pipeline(steps, verbose=True).fit_resample(X, y)

new_X.shape, new_y.shape

In [ ]:
vp.value_count(new_y.to_frame(), 'target')

In [ ]:
fig = px.scatter(pd.concat([new_X, new_y.astype(str)], axis=1).sort_values(by='target'),
                 x='feature_1', y='feature_2', color='target')

data_groups.append(fig['data'])
subplot_titles.append(f'SMOTE+Tomek {class_ratio(new_y)}')

### 3.3. ADASYN+ENN

In [ ]:
from imblearn.over_sampling import ADASYN
from imblearn.under_sampling import EditedNearestNeighbours

In [ ]:
steps = [
    ('standard_scaler', DFStandardScaler()),
    ('ADASYN', ADASYN(random_state=0, n_jobs=-1)),
    ('ENN', EditedNearestNeighbours(n_jobs=-1, sampling_strategy='all')),
]
new_X, new_y = Pipeline(steps, verbose=True).fit_resample(X, y)

new_X.shape, new_y.shape

In [ ]:
vp.value_count(new_y.to_frame(), 'target')

In [ ]:
fig = px.scatter(pd.concat([new_X, new_y.astype(str)], axis=1).sort_values(by='target'),
                 x='feature_1', y='feature_2', color='target')

data_groups.append(fig['data'])
subplot_titles.append(f'ADASYN+ENN {class_ratio(new_y)}')

### 3.4. ADASYN+Tomek

In [ ]:
from imblearn.over_sampling import ADASYN
from imblearn.under_sampling import TomekLinks

In [ ]:
steps = [
    ('standard_scaler', DFStandardScaler()),
    ('ADASYN', ADASYN(random_state=0, n_jobs=-1)),
    ('Tomek', TomekLinks(n_jobs=-1, sampling_strategy='all')),
]
new_X, new_y = Pipeline(steps, verbose=True).fit_resample(X, y)

new_X.shape, new_y.shape

In [ ]:
vp.value_count(new_y.to_frame(), 'target')

In [ ]:
fig = px.scatter(pd.concat([new_X, new_y.astype(str)], axis=1).sort_values(by='target'),
                 x='feature_1', y='feature_2', color='target')

data_groups.append(fig['data'])
subplot_titles.append(f'ADASYN+Tomek {class_ratio(new_y)}')

### 3.5. Borderline SMOTE+ENN

In [ ]:
from imblearn.over_sampling import BorderlineSMOTE
from imblearn.under_sampling import EditedNearestNeighbours

In [ ]:
steps = [
    ('standard_scaler', DFStandardScaler()),
    ('BorderlineSMOTE', BorderlineSMOTE(random_state=0, n_jobs=-1)),
    ('ENN', EditedNearestNeighbours(n_jobs=-1, sampling_strategy='all')),
]
new_X, new_y = Pipeline(steps, verbose=True).fit_resample(X, y)

new_X.shape, new_y.shape

In [ ]:
vp.value_count(new_y.to_frame(), 'target')

In [ ]:
fig = px.scatter(pd.concat([new_X, new_y.astype(str)], axis=1).sort_values(by='target'),
                 x='feature_1', y='feature_2', color='target')

data_groups.append(fig['data'])
subplot_titles.append(f'Borderline SMOTE+ENN {class_ratio(new_y)}')

### 3.6. Borderline SMOTE+Tomek

In [ ]:
from imblearn.over_sampling import BorderlineSMOTE
from imblearn.under_sampling import TomekLinks

In [ ]:
steps = [
    ('standard_scaler', DFStandardScaler()),
    ('BorderlineSMOTE', BorderlineSMOTE(random_state=0, n_jobs=-1)),
    ('Tomek', TomekLinks(n_jobs=-1, sampling_strategy='all')),
]
new_X, new_y = Pipeline(steps, verbose=True).fit_resample(X, y)

new_X.shape, new_y.shape

In [ ]:
vp.value_count(new_y.to_frame(), 'target')

In [ ]:
fig = px.scatter(pd.concat([new_X, new_y.astype(str)], axis=1).sort_values(by='target'),
                 x='feature_1', y='feature_2', color='target')

data_groups.append(fig['data'])
subplot_titles.append(f'Borderline SMOTE+Tomek {class_ratio(new_y)}')

### 3.7. K-Means SMOTE+Tomek

In [ ]:
from imblearn.over_sampling import KMeansSMOTE
from imblearn.under_sampling import EditedNearestNeighbours

In [ ]:
steps = [
    ('standard_scaler', DFStandardScaler()),
    ('KMeansSMOTE', KMeansSMOTE(random_state=0, n_jobs=-1)),
    ('ENN', EditedNearestNeighbours(n_jobs=-1, sampling_strategy='all')),
]
new_X, new_y = Pipeline(steps, verbose=True).fit_resample(X, y)

new_X.shape, new_y.shape

In [ ]:
vp.value_count(new_y.to_frame(), 'target')

In [ ]:
fig = px.scatter(pd.concat([new_X, new_y.astype(str)], axis=1).sort_values(by='target'),
                 x='feature_1', y='feature_2', color='target')

data_groups.append(fig['data'])
subplot_titles.append(f'K-Means SMOTE+ENN {class_ratio(new_y)}')

### 3.8. K-Means SMOTE+Tomek

In [ ]:
from imblearn.over_sampling import KMeansSMOTE
from imblearn.under_sampling import TomekLinks

In [ ]:
steps = [
    ('standard_scaler', DFStandardScaler()),
    ('KMeansSMOTE', KMeansSMOTE(random_state=0, n_jobs=-1)),
    ('Tomek', TomekLinks(n_jobs=-1, sampling_strategy='all')),
]
new_X, new_y = Pipeline(steps, verbose=True).fit_resample(X, y)

new_X.shape, new_y.shape

In [ ]:
vp.value_count(new_y.to_frame(), 'target')

In [ ]:
fig = px.scatter(pd.concat([new_X, new_y.astype(str)], axis=1).sort_values(by='target'),
                 x='feature_1', y='feature_2', color='target')

data_groups.append(fig['data'])
subplot_titles.append(f'K-Means SMOTE+Tomek {class_ratio(new_y)}')

### 3.9. SVM SMOTE+ENN

In [ ]:
from imblearn.over_sampling import SVMSMOTE
from imblearn.under_sampling import EditedNearestNeighbours

In [ ]:
steps = [
    ('standard_scaler', DFStandardScaler()),
    ('SVMSMOTE', SVMSMOTE(random_state=0, n_jobs=-1)),
    ('ENN', EditedNearestNeighbours(n_jobs=-1, sampling_strategy='all')),
]
new_X, new_y = Pipeline(steps, verbose=True).fit_resample(X, y)

new_X.shape, new_y.shape

In [ ]:
vp.value_count(new_y.to_frame(), 'target')

In [ ]:
fig = px.scatter(pd.concat([new_X, new_y.astype(str)], axis=1).sort_values(by='target'),
                 x='feature_1', y='feature_2', color='target')

data_groups.append(fig['data'])
subplot_titles.append(f'SVM SMOTE+ENN {class_ratio(new_y)}')

### 3.10. SVM SMOTE+Tomek

In [ ]:
from imblearn.over_sampling import SVMSMOTE
from imblearn.under_sampling import TomekLinks

In [ ]:
steps = [
    ('standard_scaler', DFStandardScaler()),
    ('SVMSMOTE', SVMSMOTE(random_state=0, n_jobs=-1)),
    ('Tomek', TomekLinks(n_jobs=-1, sampling_strategy='all')),
]
new_X, new_y = Pipeline(steps, verbose=True).fit_resample(X, y)

new_X.shape, new_y.shape

In [ ]:
vp.value_count(new_y.to_frame(), 'target')

In [ ]:
fig = px.scatter(pd.concat([new_X, new_y.astype(str)], axis=1).sort_values(by='target'),
                 x='feature_1', y='feature_2', color='target')

data_groups.append(fig['data'])
subplot_titles.append(f'SVM SMOTE+Tomek {class_ratio(new_y)}')

In [ ]:
# Scatter plot
vp.datagroups_subplots(
    data_groups,
    max_col=3,
    title='Combination-Sampling',
    out_path=OUT_PATH_GRAPH,
    subplot_kwargs={
        'subplot_titles': subplot_titles,
        'vertical_spacing': .07
    },
    layout_kwargs={'height': 1250})